In [ ]:
import nbpresent

nbpresent.__version__

from IPython.display import Image

%matplotlib inline

# Search with Inference

## Matthew A. Snell
### GA SG Data Science 5 - Project


#### 2017-10-21


# Objective

Leverage Ngrams plus TensorFlow (Word2Vec) to rank a body of *sources* against a provided search *phrase*

  - Rank Sources using Ngram based scoring (width of 3)
    
  - Suppliment Ngram search with Word2Vec (width of 5) nearest neighbours based *re-phrasing* of search term

# Lessons Learnt ... *So Far* ...

  - Who needs classes? It's just a proof of concept

    **Wrong** - Code Complexity
    
  - Who needs a DBMS? It's just a proof of concept

    **Wrong** - Performance
      
  - ETL (cleansing) is slow (cpu time)

    **Get the Data Models Right** - Re-Runs & Validators are expensive
    
  - Dictionaries `dict()` or `{ k:v }` are Great!
   
   **Nested Dictionaries are (code) Messy**:
   
   `dict = { k1: { k2: { k3: v }}}` -> `dict[k1][k2][k3]`
     
  - I'll build a non/semi-Data Science model 1st for comparision

    **Time Consuming** - More about Python than Data Science


# What is an Ngram?

For the purposes of Data Science or NLP, an Ngram (monogram, bigram, trigram, quadgram etc) is a *sliding* group of *words* taken from a set of text.

eg. `consider this line of text`

    1gram = `[consider, this, line, of, text]` - 5 elements

    2gram = `[consider this, this line, line of, of text]` - 4 elements

    3gram = `[consider this line, this line of, line of text]` - 3 elements


# But for Search Phases,
# We handle Ngrams differently

How a search phrase is sequenced or weighted by a user can be very different compared to how the terms are used.

Consider:

Search Phrase: `Python Learning Examples SciKit Data Science`

vs
         
Book Title: `Data Science SciKit Learning Python Examples`

Using Ngrams, the best hit we get beyond 1gram (low scoring), is the 2gram `Data Science` - in a sea of documents, this will not rank high - unless a word is rare eg `skikit`


To counter this, for search phrases, we generate every possible Ngram, to given width, and drop all those never seen before (ie. not in our Ngram Datastore)

eg: `Python Learning Examples SciKit Data Science` ->

`[ TODO ]`


# Datastores

### Config
Contains application wide baselines (eg. Instance UUID, Corpus Directory, Ngram width, Word2Vec Ngram Width) & master keys (currently only last used `vector`)

### Dictionary
`word` to `vector` (`int`) mappings
`vector` assigned sequentially when new `word` is found

### Vector
`vector`  (`int`) to `word` mappings

### NGram
Per `ngram`, per `srcID` list of lines with Ngram


### Sources
Per Source Path (file path, URL etc) to `srcID` (UUID)

Potential to add source has (ND5 or SHA1) to avoid duplicates

### DocMeta (Document Meta)
Per `source` summary information, including type (URL, FILE, ...) and sub-type (HTML, PDF, TXT, CSV, ...), index status, Word2Vec status, `list` of ngrams, indexed data version, when indexed etc

### DocStat (Document Statistics)
Per document (`srcID`), per line list of Ngrams


# Algorithm - Index - Ngram


# Algorithm - Word2Vec

# Algorithm - Search